# Cálculo de deducciones incrementadas de Tramo 1 y Tramo 2

## A. Importación de módulos y librerías

In [9]:
import pandas as pd
import numpy as np

from remuneraciones.prorrateos import *
from parametrizacion.conceptos import ParamConceptos
from deducciones.incremental.tablas_tramo2 import *

from helpers.bases import tabla
from helpers.progressbar import ProgressBar
from helpers.dict import unpack

from data import CWTR_SUM, DOTACION, CUILES, F572_WEB

## B. Parametrización de conceptos

In [116]:
conceptos = ParamConceptos()
conceptos

## C. Deducción incrementada Tramo 1

### 1. Unión de los orígenes de datos

#### CWTR

In [117]:
cwtr = CWTR_SUM[['legajo', 'CCn', 'mes', 'importe']].copy()
cwtr.rename(columns={'CCn': 'codigo'}, inplace=True)

cwtr

,legajo,codigo,mes,importe
0,6000002,/111,4,14044.00
1,6000002,/111,6,308968.00
2,6000002,/111,8,16572.00
3,6000002,/111,1,18317.99
4,6000002,/111,1,305300.00
...,...,...,...,...
483551,6114233,/361,12,1219.27
483552,6114235,/351,12,1309.09
483553,6114235,/361,12,1309.09
483554,6114237,/351,12,1777.31


#### F.572 Web

In [118]:
f572 = F572_WEB.copy()
f572['cuil'] = f572['cuil'].astype('int64')
f572 = f572.merge(CUILES, how='left', on='cuil')
f572 = f572[['legajo', 'codigo', 'mes', 'importe']]
f572.dropna(inplace=True)

f572

,legajo,codigo,mes,importe
0,6106059.0,Q-Deduc-1,7,4380.80
1,6106059.0,Q-Deduc-2,13,3962.89
2,6106059.0,Q-Deduc-22,12,18000.00
3,6071357.0,Q-RetPerPagCu-27,13,113153.25
4,6071357.0,Q-Deduc-22,1,8400.00
...,...,...,...,...
77554,6002316.0,Q-Deduc-1,8,15992.93
77555,6002316.0,Q-Deduc-1,9,11015.22
77556,6002316.0,Q-Deduc-1,10,15526.32
77557,6002316.0,Q-Deduc-1,11,15606.62


#### Prorrateos

In [119]:
tramos = tabla('bases_tramos')
legajos_meses_tramo1 = tramos[tramos['tramo'].eq(1)]
prorrateados = Prorrateos().prorrateados()

prorrateos_aplicables = prorrateados.merge(legajos_meses_tramo1, how='inner', left_on=['legajo', 'mes_pago'], right_on=['legajo', 'mes'])
prorrateos = prorrateos_aplicables[['legajo', 'CCn', 'mes_x', 'importe_x']].copy()
prorrateos.rename(columns={'CCn': 'codigo', 'mes_x': 'mes', 'importe_x': 'importe'}, inplace=True)

prorrateos

,legajo,codigo,mes,importe
0,6000009,#1256,6,713.195714
1,6000009,#1256,7,713.195714
2,6000009,#1256,8,713.195714
3,6000009,#1256,9,713.195714
4,6000009,#1256,10,713.195714
...,...,...,...,...
40817,6106947,#1256,12,706.000000
40818,6106947,#1258,10,0.000000
40819,6106947,#1258,11,0.000000
40820,6106947,#1258,12,0.000000


#### Unión

In [120]:
total_conceptos = pd.concat([cwtr, f572, prorrateos])
assert len(cwtr) + len(f572) + len(prorrateos) == len(total_conceptos), 'No coinciden las sumas!'

total_conceptos

,legajo,codigo,mes,importe
0,6000002.0,/111,4,14044.00
1,6000002.0,/111,6,308968.00
2,6000002.0,/111,8,16572.00
3,6000002.0,/111,1,18317.99
4,6000002.0,/111,1,305300.00
...,...,...,...,...
40817,6106947.0,#1256,12,706.00
40818,6106947.0,#1258,10,0.00
40819,6106947.0,#1258,11,0.00
40820,6106947.0,#1258,12,0.00


### 2. Filtrado de conceptos aplicables

In [121]:
# Control
conceptos_parametrizados = list(conceptos.data['CCn'].unique())

for c in list(total_conceptos['codigo'].unique()):
    if c not in conceptos_parametrizados:
        raise NotImplementedError(f'No se ha definido parametría para el concepto: {c}')

In [122]:
df = total_conceptos
df['legajo'] = df['legajo'].astype('int64')

df['Suma'] = np.where(df['codigo'].isin(conceptos.listar('DedIncT1', '+')), df['importe'], 0)
df['Resta'] = np.where(df['codigo'].isin(conceptos.listar('DedIncT1', '-')), df['importe'], 0)
df['DeduccIncremT1'] = df['Suma'] - df['Resta']

# Se filtran cuando sólo sean tramo 1
df = df.merge(legajos_meses_tramo1, how='inner', on=['legajo', 'mes'])

In [ ]:
deduc_incr_tramo1_mensual = df.groupby(['legajo','mes'])['DeduccIncremT1'].sum().reset_index()
deduc_incr_tramo1_anual = df.groupby(['legajo'])['DeduccIncremT1'].sum().reset_index()

deduc_incr_tramo1_mensual.to_excel('prueba.xlsx')

## D. Deducción incrementada Tramo 2

In [125]:
legajos_meses_tramo2 = tramos[tramos['tramo'].eq(2)]
legajos_meses_tramo2

,legajo,mes,importe,promedio,base_tramo,tramo
15,6000009.0,4.0,230493.70,232092.88,230493.70,2
16,6000009.0,5.0,246092.17,234892.74,234892.74,2
18,6000009.0,7.0,298932.97,283397.79,283397.79,2
19,6000009.0,8.0,351847.36,306214.31,306214.31,2
22,6000009.0,11.0,429796.04,429796.04,429796.04,2
...,...,...,...,...,...,...
35807,6111351.0,12.0,392871.33,333819.76,333819.76,2
35831,6111355.0,12.0,525739.87,360098.82,360098.82,2
35939,6111382.0,12.0,502104.76,332666.32,332666.32,2
35951,6111383.0,12.0,586905.51,390407.44,390407.44,2


In [126]:
tabla('tablas_afip_tramo2')

,base,deduccion,ini,fin
0,226087,64737.0,1,5
1,226238,64180.0,1,5
2,226389,63669.0,1,5
3,226539,63187.0,1,5
4,226690,62726.0,1,5
...,...,...,...,...
687,429882,2534.0,11,12
688,430321,2005.0,11,12
689,430761,1476.0,11,12
690,431201,947.0,11,12


In [127]:
deducc_tramo2 = []

p = ProgressBar(len(legajos_meses_tramo2))
for i, row in legajos_meses_tramo2.iterrows():

    legajo, base, mes = unpack(row, 'legajo', 'base_tramo', 'mes')

    deduccion = TablasTramo2.deduccion(mes, base)
    deducc_tramo2.append({
        'legajo':           legajo, 
        'mes':              mes, 
        'DeduccIncremT2':   deduccion
    })
    p.next()

deducc_tramo2 = pd.DataFrame.from_records(deducc_tramo2)
deducc_tramo2

|██████████████████████████████████████████████████| 100.0 %


,legajo,mes,DeduccIncremT2
0,6000009.0,4.0,53051.00
1,6000009.0,5.0,43997.00
2,6000009.0,7.0,100180.08
3,6000009.0,8.0,39686.27
4,6000009.0,11.0,2534.00
...,...,...,...
6415,6111351.0,12.0,122359.00
6416,6111355.0,12.0,85812.00
6417,6111382.0,12.0,123863.00
6418,6111383.0,12.0,37870.00


## E. Unificación con base tramos

In [128]:
tramos

,legajo,mes,importe,promedio,base_tramo,tramo
0,6000002.0,1.0,466593.92,466593.92,466593.92,3
1,6000002.0,2.0,1405581.67,936087.80,936087.80,3
2,6000002.0,3.0,359751.07,743975.55,359751.07,3
3,6000002.0,4.0,363686.19,648903.21,363686.19,3
4,6000002.0,5.0,363361.69,591794.91,363361.69,3
...,...,...,...,...,...,...
37171,6114237.0,8.0,0.00,0.00,0.00,1
37172,6114237.0,9.0,0.00,0.00,0.00,1
37173,6114237.0,10.0,0.00,0.00,0.00,1
37174,6114237.0,11.0,0.00,0.00,0.00,1


In [129]:
deduc_incr_tramo1_mensual

,legajo,mes,DeduccIncremT1
0,6000002,7,151063.280000
1,6000009,2,161553.570000
2,6000009,3,151520.420000
3,6000009,6,200913.285714
4,6000013,6,197546.540000
...,...,...,...
15090,6114229,12,-2829.820000
15091,6114231,12,-2829.820000
15092,6114233,12,-2438.540000
15093,6114235,12,35021.750000


In [130]:
con_tramo1 = tramos.merge(deduc_incr_tramo1_mensual, how='left', on=['legajo', 'mes'])
con_tramo2 = con_tramo1.merge(deducc_tramo2, how='left', on=['legajo', 'mes'])

In [132]:
con_tramo2.to_excel('deducciones_incrementadas.xlsx')

In [2]:
DOTACION

,legajo,nombre,provincia,contrato,fecha_ingreso,fecha_antig_reconoc,fecha_baja
0,6000002,"SOSA,JULIETA VERONICA",Córdoba,Ilimitado AR,2007-10-08,2007-10-08,2262-04-11 23:47:16.854775807
1,6000009,"MARTO,IVAN ANDRES",Córdoba,Ilimitado AR,2015-06-19,2012-03-12,2262-04-11 23:47:16.854775807
2,6000010,"MONTALVAN,ROLANDO",Córdoba,Ilimitado AR,1991-11-05,1991-11-05,2262-04-11 23:47:16.854775807
3,6000011,"BORTOLI,IGNACIO DARIO",Santa Fe,Ilimitado AR,2001-08-02,2001-08-02,2262-04-11 23:47:16.854775807
4,6000013,"ODETTI,LUCIANA",Córdoba,Ilimitado AR,2007-01-22,2007-01-22,2262-04-11 23:47:16.854775807
...,...,...,...,...,...,...,...
3093,6113602,"MARRERO,JOAQUIN GUILLERMO",Andina Bs As,Trab.temporada AR,2023-01-10,2022-12-14,2023-02-14 00:00:00.000000000
3094,6039084,"FERREYRA,RODRIGO EZEQUIEL",Córdoba,Trab.temporada AR,2023-01-02,2020-10-06,2262-04-11 23:47:16.854775807
3095,6058106,"CARRIZO,MARTIN RICARDO",Córdoba,Trab.temporada AR,2023-02-06,2022-09-24,2262-04-11 23:47:16.854775807
3096,6106640,"DOMINGUEZ,FACUNDO",Andina Bs As,Trab.temporada AR,2023-03-20,2022-09-10,2262-04-11 23:47:16.854775807


In [8]:
minimo_no_imponible = {
    'normal': {
        'mni': 252_564.84,
        'esp': 1_212_311.24
    },
    'especial': {
        'mni': 308_129.12,
        'esp': 1_479_019.72
    }
}

provincias_especiales = [
    'Neuquén',
    'Chubut',
    'Río Negro',
    'Tierra del Fuego',
    'Santa Cruz',
    'La Pampa',
]

In [23]:
prov_dotacion = list(DOTACION['provincia'].unique())

for p in provincias_especiales:
    if p not in prov_dotacion:
        raise NotImplementedError(f'No hay empleados en la provincia: {p}')

In [32]:
p = ProgressBar(len(DOTACION))
count = 1
for i, row in DOTACION.iterrows():


    count += 1
    if count == 200:
        break

    
    legajo, provincia = unpack(row, 'legajo', 'provincia')

    tipo_mni = 'especial' if provincia in provincias_especiales else 'normal'

    deducciones = minimo_no_imponible[tipo_mni]

    print(deducciones['mni'], deducciones['esp'])
    
    # p.next()


252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 1212311.24
252564.84 121